In [1]:
import numpy as np
import pandas as pd
import keras

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Multiple_Forms.csv', index_col = 0)
df.head()

,Author,Poem,Form
0,William Shakespeare,"From fairest creatures we desire increase,\r\n...",Sonnet
1,William Shakespeare,"When forty winters shall besiege thy brow,\r\n...",Sonnet
2,William Shakespeare,Look in thy glass and tell the face thou viewe...,Sonnet
3,William Shakespeare,"Unthrifty loveliness, why dost thou spend\r\n ...",Sonnet
4,William Shakespeare,"Those hours, that with gentle work did frame\r...",Sonnet


In [3]:
poems = df['Poem']
poems

0     From fairest creatures we desire increase,\r\n...
1     When forty winters shall besiege thy brow,\r\n...
2     Look in thy glass and tell the face thou viewe...
3     Unthrifty loveliness, why dost thou spend\r\n ...
4     Those hours, that with gentle work did frame\r...
                            ...                        
95    Author Note: Divers Princes and Noblemen being...
96    Kynge Jamy, Jomy your joye is all go.\nYe summ...
97    A street there is in Paris famous, \nFor which...
98    Ye shepherds give ear to my lay, \nAnd take no...
99    It was our war-ship ~Clampherdown~\n Would swe...
Name: Poem, Length: 537, dtype: object

In [4]:
text = ''
for poem in poems:
    text += poem
len(text)

829989

In [5]:
maxlen = 60                                                            
step = 3                                                               

sentences = []                                                         

next_chars = []                                                        

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))                                        
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)       

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)      
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)              
for i, sentence in enumerate(sentences):                               
    for t, char in enumerate(sentence):                                
        x[i, t, char_indices[char]] = 1                                
    y[i, char_indices[next_chars[i]]] = 1 

Number of sequences: 276643
Unique characters: 103
Vectorization...


In [6]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [7]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
import random
import sys

for epoch in range(1, 20):                                          
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)                       
    start_index = random.randint(0, len(text) - maxlen - 1)         
    generated_text = text[start_index: start_index + maxlen]        
    print('--- Generating with seed: "' + generated_text + '"')     
    for temperature in [0.2, 0.5, 1.0, 1.2]:                        
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        for i in range(400):                                        
            sampled = np.zeros((1, maxlen, len(chars)))             
            for t, char in enumerate(generated_text):               
                sampled[0, t, char_indices[char]] = 1.              

            preds = model.predict(sampled, verbose=0)[0]            
            next_index = sample(preds, temperature)                 
            next_char = chars[next_index]                           

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)

epoch 1
Epoch 1/1
276643/276643 [==============================] - 360s 1ms/step - loss: 2.0358
--- Generating with seed: "ith the dew the yellow hawthorns bow;
The rustic whistles th"
------ temperature: 0.2
ith the dew the yellow hawthorns bow;
The rustic whistles the world the hard 
And stranger his through the sea grown
And the secring the song the sorned of the sea 
The sea where the sang the secred stranger of his sade 
And the sing the sea the world the sing the sea 
The blood of the see and the sore the sea 
And the sorned the show-shard the world the way. 



"The sing the world the sea warked and the wind
And the see the see the sea which and which th------ temperature: 0.5
 and the wind
And the see the see the sea which and which the stranged
                                                                                                                                                                                                                                            

With theederithoo own lustering of Cha
Ep? 
But never mer-; ; Black's 
Pllee if Alent with " whose flies enote thee 
Bailan ofbloo a tookers, Asfarfult follow and re and Wide
        To thenal geathoun be would plase would as thinklog, in GlifterM epoch 5
Epoch 1/1
276643/276643 [==============================] - 364s 1ms/step - loss: 1.6517
--- Generating with seed: "flooded lowlands 
And slopes of sodden loam 
The pack-horse "
------ temperature: 0.2
flooded lowlands 
And slopes of sodden loam 
The pack-horse the song the mine.

The song the sea winds the sea wind
                                                                                                                                                                                                                                                                                                                                                        ------ temperature: 0.5
                                                           

Be the milture, ; the Horse of ------ temperature: 1.2
ing eternal keet to fould? 

Be the milture, ; the Horse of sode kin.
And you cropping, I wasBeved than,
And 
Halume faulted sprisine, heavening gace, that and haw our years, 
winus, happier, heed it love,
He frowl, igla wind! One,
 alonered, ! none, 
When put hest virisgers with numbs forguln and crows, 
He !urstirus' for thou candon waste thousald man,  de't youbrobles, face rofiouchs,
Sulvius Ern eft betury
  o eyed through hafp'we"

Soon, nrat approw, epoch 9
Epoch 1/1
276643/276643 [==============================] - 353s 1ms/step - loss: 1.5981
--- Generating with seed: "ad been no such matter.
Some wits have wonder'd what analogy"
------ temperature: 0.2
ad been no such matter.
Some wits have wonder'd what analogy,
The string of the stars the strange the streame
The stream the stormed the string of the streamed 
The fall the struck the stream the sight
To the string of the strange to the struck,
The strange the sight to the s

C:\Users\kyle_\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


 glickin of friends: the vain Form a facoce was long
Who may medagatt to paiss,
By the burnte, all wuth rose true cries; 
Tands ayshi,' you ve! let behold ose.
In worn Guthuls first How Bares
 Halst utone ! Hearl.o''r mymans is arsI
T' tto a lips notered mock's,Ed, to shol thould;epoch 10
Epoch 1/1
276643/276643 [==============================] - 352s 1ms/step - loss: 1.5891
--- Generating with seed: " false in rolling,
  Gilding the object whereupon it gazeth"
------ temperature: 0.2
 false in rolling,
  Gilding the object whereupon it gazethed with the season the season 
Who save the sand the sad and state 
That the season of the sigh the last the sad was a great string 
That the sad a bright and state 
That the sand a broken and the stars,
And the stand of the season and said 
The said the head of the sand 
The sand the sea the sand 
With the sang and the string and the sea and sad 
That the sang the world the cloud the sad 
The san------ temperature: 0.5
 and sad 
That the sang the

    But cillt friend's for its as much as the sheap in comn.veuavine. . . ........ . "thou then, -get me his swin ------ temperature: 1.2
 comn.veuavine. . . ........ . "thou then, -get me his swin pels,
oo a harp not---
No.portor pain, and crie, and Glaned luck
with his man'ouuts; 
No carm bidsting; though lionestr myst brother
  midilies of close eirst so dure that heax
winw'king off.In to thateniture us of an kiss with sday,
  Is cheek permanient wfoues, tombre,
  And insome him with time is troud that carm
vainin Shifo
And I -........

I have he first angely drespfup, spare? S canes epoch 14
Epoch 1/1
276643/276643 [==============================] - 349s 1ms/step - loss: 1.5647
--- Generating with seed: "usty dreams,
And the startled birds went up in streams,
For "
------ temperature: 0.2
usty dreams,
And the startled birds went up in streams,
For the stars the sighs and stars and stars 
And the sea with the sea the stars,
And the stars and stream the said,
And the words the sacred

warrs of the skies, and brave away,
In the being thantle, watched sleep 
The world to see,
He went away because the nesse,
To famder every to first door.

For it to desad, he went shornly comblece in the fams,
And if that shed in srenting bossom be
And tone shows 
 The Biltswarchiment,
sobeled weEoguame out lample to I saw
Life weary graces with hementre slangel
unchem and the sky,
And Hilld oace on with inweniom 
with man------ temperature: 1.2
nchem and the sky,
And Hilld oace on with inweniom 
with many stromish fight, 
ohathsaC Deeis vale the Echisprprune and done
Nar-was let the fagual ;
And the Your.

For thet garmen, his and offeact, 
In Wessex with frunted living, drie!'hand,' spendion: the bhos was coran teap
That because in loverndends to thime
And more Nwil-haly was to old bridely.
ge is nee do, last see,'t with horm te!
  "Extising before rogaveas my tomb!''s manhax bits meke not, epoch 18
Epoch 1/1
276643/276643 [==============================] - 374s 1ms/step - loss: 1.54